# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
from citipy import citipy
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets
import pandas as pd
import requests
from pprint import pprint
from api_keys import weather_api_key
import random

In [4]:
def get_random_floats(low, high, k=1):
    """ Return a unique random floats
        in the range of low <= x <= high
    """
    result = []
    seen = set()
    for i in range(k):
        x=random.uniform(low, high)
        while x in seen:
            x=random.uniform(low, high)
        seen.add(x)
        result.append(x)
    return result

def get_random_lat():
    return random.uniform(-90, 90)

def get_random_lon():
    return random.uniform(-180, 180)

def generate_city_response(city_name):
    url=f"http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={weather_api_key}&units=imperial"
    return requests.get(url).json()

def print_city(num,city_name):
    print(f"Processing record {num+1)%50} of set {round((num+1)/50)+1} | {city_name.lower()}")

## Generate Cities List

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [7]:
latitudes=get_random_floats(-90,90,500)
longitudes=get_random_floats(-180,180,0)
cities_json=[]
for i in range(500):
    city=citipy.nearest_city(latitudes[i],longitudes[i])
    response_json=generate_city_response(city.city_name)
    try:
        print_city(i,response_json)
        print(f"Processing record {(i+1)%50} of set {round((i+1)/50)+1} | {response_json['name'].lower()}")
    except KeyError:
        print(f"Processing record {(i+1)%50} of set {round((i+1)/50)+1} | {city.city_name}")
        print("City not found. Skipping...")
        while response_json.get("name")==None:
            city=citipy.nearest_city(get_random_lat(),get_random_lon())       
            response_json=generate_city_response(city.city_name)
            while response_json in cities_json:
                city=citipy.nearest_city(get_random_lat(),get_random_lon())       
                response_json=generate_city_response(city.city_name)
    
    cities_json.append(response_json)

Processing record 1 of set 1 | samusu
City not found. Skipping...
Processing record 2 of set 1 | rikitea
Processing record 3 of set 1 | iqaluit
Processing record 4 of set 1 | caconda
Processing record 5 of set 1 | richards bay
Processing record 6 of set 1 | hobart
Processing record 7 of set 1 | george town
Processing record 8 of set 1 | mataura
Processing record 9 of set 1 | ribeira grande
Processing record 10 of set 1 | amarante do maranhão
Processing record 11 of set 1 | taolanaro
City not found. Skipping...
Processing record 12 of set 1 | jamestown
Processing record 13 of set 1 | ushuaia
Processing record 14 of set 1 | butaritari
Processing record 15 of set 1 | alanya
Processing record 16 of set 1 | rikitea
Processing record 17 of set 1 | puerto ayora
Processing record 18 of set 1 | illoqqortoormiut
City not found. Skipping...
Processing record 19 of set 1 | shahr-e bābak
Processing record 20 of set 1 | malanje
Processing record 21 of set 1 | hithadhoo
Processing record 22 of set 1 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [11]:
rows = [[cities_json[i]['name'],cities_json[i]['coord']['lat'],cities_json[i]['coord']['lon'],cities_json[i]['main']['temp_max'],cities_json[i]['main']['humidity'],cities_json[i]['clouds']['all'],cities_json[i]['wind']['speed'],cities_json[i]['sys']['country'],cities_json[i]['dt'] ] for i in range(500)]
df = pd.DataFrame(rows, columns=["City", "Lat","Lon","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"])
df.head()

500

In [5]:
count=df.count()
# info_df=grouped_city["Lat","Lon","Max Temp","Humidity","Cloudiness","Wind Speed","Date"].agg(["count","mean","std","min","max"])
# info_df.head()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [6]:
#  Get the indices of cities that have humidity over 100%.


In [7]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression